In [423]:
import requests
import pandas as pd
import csv

In [424]:
def fetch():
    molde = "lat=62.73752&lon=7.15912"
    url = f"https://api.met.no/weatherapi/locationforecast/2.0/complete?{molde}"
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.81 Safari/537.36"
    }

    response = requests.get(url, headers=headers)
    return response.json()


data = fetch()

In [425]:
from datetime import datetime
import os.path

now = datetime.now().strftime("%Y-%m-%dT%H:00:00Z")
data = data["properties"]["timeseries"]

for entry in data:
    if entry["time"] == now:
        data = entry["data"]["instant"]["details"]
        date, time = now.replace("Z", "").split("T")
        data["time"] = time
        data["date"] = date
        cols = [col for col in data.keys()]

# Check if file exists in order to avoid writing header multiple times
file_exists = os.path.isfile("test.csv")

with open("test.csv", "a", newline="") as file:
    writer = csv.DictWriter(file, fieldnames=cols)

    if not file_exists:
        writer.writeheader()

    writer.writerow(data)

In [428]:
df = pd.read_csv("test.csv")

# Change order of columns, setting time and date first.
cols = df.columns.tolist()
cols = cols[-2:] + cols[:-2]
df = df[cols]

df = df.drop_duplicates()
df


,time,date,air_pressure_at_sea_level,air_temperature,air_temperature_percentile_10,air_temperature_percentile_90,cloud_area_fraction,cloud_area_fraction_high,cloud_area_fraction_low,cloud_area_fraction_medium,dew_point_temperature,fog_area_fraction,relative_humidity,ultraviolet_index_clear_sky,wind_from_direction,wind_speed,wind_speed_of_gust,wind_speed_percentile_10,wind_speed_percentile_90
0,15:00:00,2021-10-27,998.6,11.7,10.9,12.4,48.8,0.0,46.9,2.2,7.5,0.0,75.3,0.0,253.4,4.3,7.1,4.0,5.4
3,15:00:00,2021-10-27,998.6,11.9,11.1,12.6,48.8,0.0,46.9,2.2,7.6,0.0,75.3,0.0,253.4,4.3,7.1,4.0,5.4
